In [1]:
import os
import numpy as np 
import pandas as pd

In [2]:
path = r"C:\Users\piush\Desktop\World Bank Poverty\submissions\b_sub"

all_files = os.listdir(path)

# Read and concatenate submissions
outs = [pd.read_csv(os.path.join(path, f), index_col=[0,1]) for f in all_files]
concat_sub = pd.concat(outs, axis=1)
cols = list(map(lambda x: "is_poor_" + str(x), range(len(concat_sub.columns))))
concat_sub.columns = cols
concat_sub.reset_index(inplace=True)
concat_sub.head()

,id,country,is_poor_0,is_poor_1,is_poor_2,is_poor_3,is_poor_4,is_poor_5,is_poor_6,is_poor_7,is_poor_8,is_poor_9,is_poor_10,is_poor_11,is_poor_12
0,9135,B,0.008006,0.003316,0.001767,0.002474,0.008006,0.002684,0.002953,0.002700,0.003467,0.007510,0.005310,0.004355,0.008006
1,117,B,0.069620,0.057328,0.034412,0.062617,0.069620,0.024344,0.063372,0.050885,0.027196,0.009448,0.061243,0.059690,0.069620
2,29085,B,0.003793,0.002233,0.005325,0.001520,0.003793,0.010770,0.000537,0.004351,0.001102,0.002976,0.003098,0.001695,0.003793
3,55442,B,0.082591,0.015402,0.151245,0.060690,0.082591,0.028010,0.113654,0.103925,0.071562,0.091118,0.048779,0.044030,0.082591
4,29281,B,0.001408,0.010322,0.004402,0.001564,0.001408,0.036404,0.008339,0.007451,0.002548,0.010373,0.005592,0.001854,0.001408


In [3]:
concat_sub.iloc[:,2:15].head(2)

,is_poor_0,is_poor_1,is_poor_2,is_poor_3,is_poor_4,is_poor_5,is_poor_6,is_poor_7,is_poor_8,is_poor_9,is_poor_10,is_poor_11,is_poor_12
0,0.008006,0.003316,0.001767,0.002474,0.008006,0.002684,0.002953,0.002700,0.003467,0.007510,0.005310,0.004355,0.008006
1,0.069620,0.057328,0.034412,0.062617,0.069620,0.024344,0.063372,0.050885,0.027196,0.009448,0.061243,0.059690,0.069620


In [5]:
concat_sub['poor'] = concat_sub.iloc[:,2:15].mean(axis = 1)

In [6]:
b = concat_sub[['id','country','poor']]

In [7]:
b.to_csv("b_1.csv")

In [ ]:
# get the data fields ready for stacking
concat_sub['is_poor_max'] = concat_sub.iloc[:, 2:18].max(axis=1)
concat_sub['is_poor_min'] = concat_sub.iloc[:, 2:18].min(axis=1)
concat_sub['is_poor_mean'] = concat_sub.iloc[:, 2:18].mean(axis=1)
concat_sub['is_poor_median'] = concat_sub.iloc[:, 2:18].median(axis=1)

In [ ]:
# set up cutoff threshold for lower and upper bounds, easy to twist 
cutoff_lo = 0.8
cutoff_hi = 0.2

In [ ]:
concat_sub.head()

### Change the column names 

###### Mean Stacking

In [ ]:
concat_sub['is_poor'] = concat_sub['is_poor_mean']
concat_sub[['id','country', 'is_poor']].to_csv('stack_mean.csv', 
                                        index=False, float_format='%.6f')

###### Median Stacking

In [ ]:
concat_sub['is_poor'] = concat_sub['is_poor_median']
concat_sub[['id','country', 'is_poor']].to_csv('stack_median.csv', 
                                        index=False, float_format='%.6f')

###### PushOut + Median Stacking
Pushout strategy is a bit agressive given what it does...

In [ ]:
concat_sub['is_poor'] = np.where(np.all(concat_sub.iloc[:,1:6] > cutoff_lo, axis=1), 1, 
                                    np.where(np.all(concat_sub.iloc[:,1:6] < cutoff_hi, axis=1),
                                             0, concat_sub['is_poor_median']))
concat_sub[['id','country', 'is_poor']].to_csv('stack_pushout_median.csv', 
                                        index=False, float_format='%.6f')

###### MinMax + Mean Stacking
MinMax seems more gentle and it outperforms the previous one given its peformance score.

In [ ]:
concat_sub['is_poor'] = np.where(np.all(concat_sub.iloc[:,1:6] > cutoff_lo, axis=1), 
                                    concat_sub['is_poor_max'], 
                                    np.where(np.all(concat_sub.iloc[:,1:6] < cutoff_hi, axis=1),
                                             concat_sub['is_poor_min'], 
                                             concat_sub['is_poor_mean']))
concat_sub[['id','country', 'is_poor']].to_csv('stack_minmax_mean.csv', 
                                        index=False, float_format='%.6f')

###### MinMax + Median Stacking

In [ ]:
concat_sub['is_poor'] = np.where(np.all(concat_sub.iloc[:,1:6] > cutoff_lo, axis=1), 
                                    concat_sub['is_poor_max'], 
                                    np.where(np.all(concat_sub.iloc[:,1:6] < cutoff_hi, axis=1),
                                             concat_sub['is_poor_min'], 
                                             concat_sub['is_poor_median']))
concat_sub[['id','country', 'is_poor']].to_csv('stack_minmax_median.csv', 
                                        index=False, float_format='%.6f')

###### MinMax + BestBase Stacking

In [ ]:
# load the model with best base performance
sub_base = pd.read_csv(r"C:\Users\pvaish10\Desktop\csv_poverty\submission_X_G_tr2.csv")

In [ ]:
concat_sub['is_poor_base'] = sub_base['poor']
concat_sub['is_poor'] = np.where(np.all(concat_sub.iloc[:,1:6] > cutoff_lo, axis=1), 
                                    concat_sub['is_poor_max'], 
                                    np.where(np.all(concat_sub.iloc[:,1:6] < cutoff_hi, axis=1),
                                             concat_sub['is_poor_min'], 
                                             concat_sub['is_poor_base']))
concat_sub[['id','country', 'is_poor']].to_csv('stack_minmax_bestbase.csv', 
                                        index=False, float_format='%.6f')